In [ ]:
# @title Imports
import requests
import base64
import pandas as pd
from google.colab import drive
import time

# Introduction

This is a test to see changes

In [ ]:
# @title Ingest Data from Drive
drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/Most Streamed Spotify Songs 2024.csv',encoding='ISO-8859-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Get Track/Artist Pairs for Searching
pairs = [x for x in zip(df['Track'], df['Artist'])]

In [ ]:
# @title Access Spotify API
# Read the client secret from a file
with open('/content/drive/MyDrive/client_secret.txt', 'r') as file:
    client_secret = file.read().strip()  # Removes any extraneous whitespace or newlines

# Read the client id from a file
with open('/content/drive/MyDrive/client_id.txt', 'r') as file:
    client_id = file.read().strip()  # Removes any extraneous whitespace or newlines

auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

headers = {
    "Authorization": f"Basic {b64_auth_str}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
access_token = response.json()['access_token']


search_url = 'https://api.spotify.com/v1/search'
query = 'track:Me Acostumbre (feat. Bad Bunny) artist:Arcï¿½ï¿½'  # Customize as needed

headers = {
    'Authorization': f'Bearer {access_token}'
}
params = {
    'q': query,
    'type': 'track',
    'limit': 1
}


In [ ]:

# Store the results in a dictionary
track_ids = {}
failed=[]
num=0
# Loop through each song
for track, artist in pairs:
    num+=1
    # Prepare search query
    query = f"track:{track} artist:{artist}"
    search_url = "https://api.spotify.com/v1/search"
    params = {
        'q': query,
        'type': 'track',
        'limit': 1
    }

    # Make the request
    response = requests.get(search_url, headers=headers, params=params)
    result = response.json()

    # Extract track ID if found
    if result['tracks']['items']:
        track_id = result['tracks']['items'][0]['id']
        track_ids[(track, artist)] = (track_id,'track/artist')
        print(f"ID found for {track} by {artist} ({num})")

    else:
        print(f"No ID found for {track} by {artist}")
        # Prepare search query
        query = f"track:{track}"
        search_url = "https://api.spotify.com/v1/search"
        params = {
            'q': query,
            'type': 'track',
            'limit': 1
        }

        # Make the request
        response = requests.get(search_url, headers=headers, params=params)
        result = response.json()

        # Extract track ID if found
        if result['tracks']['items']:
            track_id = result['tracks']['items'][0]['id']
            track_ids[(track, artist)] = (track_id,'track')
            print(f"ID found for {track} by searching on title only ({num})")
        else:
            print(f"No ID found for {track} by searching on title only")
            failed.append((track, artist))
    # Pause to avoid rate limiting
    time.sleep(.2)  # Adjust as needed to stay within rate limits

print("All track IDs:", track_ids)

Streaming output truncated to the last 5000 lines.
ID found for Eyes Closed by Ed Sheeran (168)
ID found for I Remember Everything (feat. Kacey Musgraves) by Zach Bryan (169)
ID found for Circles by Post Malone (170)
ID found for EL AMOR DE SU VIDA by Grupo Frontera (171)
ID found for About Damn Time by Lizzo (172)
ID found for Closer by The Chainsmokers (173)
ID found for FLOWER by JISOO (174)
ID found for PERRO NEGRO by Bad Bunny (175)
ID found for Rich Baby Daddy (feat. Sexyy Red & SZA) by Drake (176)
ID found for Havana (feat. Young Thug) by Camila Cabello (177)
ID found for LUNA by Feid (178)
ID found for I Like You (A Happier Song) (with Doja Cat) by Post Malone (179)
ID found for Bones by Imagine Dragons (180)
No ID found for Segï¿½ï¿½n Qu by Maluma
No ID found for Segï¿½ï¿½n Qu by searching on title only
ID found for Selfish by Justin Timberlake (182)
ID found for golden hour by JVKE (183)
ID found for Levitating (feat. DaBaby) by Dua Lipa (184)
ID found for Get It Sexyy by Sex

In [ ]:
!touch /content/drive/MyDrive/track_ids.txt

In [ ]:
with open('/content/drive/MyDrive/track_ids.txt', 'w') as file:
    for key, value in track_ids.items():
        file.write(f"{key}: {value}\n")

In [ ]:
!touch /content/drive/MyDrive/failed_track_ids.txt

In [ ]:
with open('/content/drive/MyDrive/failed_track_ids.txt', 'w') as file:
    for key, value in failed:
        file.write(f"{key}: {value}\n")

In [ ]:
tracks_dict = {}
for (song,artist),(id,search_type) in [i for i in track_ids.items()]:
  tracks_dict[id] = {}
  tracks_dict[id]['title'] = song
  tracks_dict[id]['artist'] = artist
  tracks_dict[id]['search_type'] = search_type

In [ ]:
tracks_df = pd.DataFrame.from_dict(tracks_dict).T

In [ ]:
existing_audio_features = pd.read_csv('/content/drive/MyDrive/audio_features_by_track.csv')

In [ ]:
existing_audio_features.set_index('Unnamed: 0',inplace=True)

In [ ]:
final_tracks_df = tracks_df.set_index('Unnamed: 0').combine_first(existing_audio_features).reset_index()
final_tracks_df

# tracks_df = pd.concat([tracks_df,
#            existing_audio_features[[col for col in list(existing_audio_features) if col not in list(tracks_df)]]
#            ],axis=1)

,Unnamed: 0,acousticness,analysis_url,artist,danceability,duration_ms,energy,id,instrumentalness,key,...,mode,search_type,speechiness,tempo,time_signature,title,track_href,type,uri,valence
0,003vvx7Niy0yvhvHt4a68B,0.00121,https://api.spotify.com/v1/audio-analysis/003v...,The Killers,0.352,222973.0,0.9110,003vvx7Niy0yvhvHt4a68B,0.000000,1.0,...,1.0,track/artist,0.0747,148.033,4.0,Mr. Brightside,https://api.spotify.com/v1/tracks/003vvx7Niy0y...,audio_features,spotify:track:003vvx7Niy0yvhvHt4a68B,0.236
1,006oGnrSZevqZTimsD2oh4,0.32900,https://api.spotify.com/v1/audio-analysis/006o...,Melike ï¿½ï¿½,0.616,259704.0,0.8820,006oGnrSZevqZTimsD2oh4,0.000002,5.0,...,1.0,track,0.0894,134.046,4.0,Diva Yorgun,https://api.spotify.com/v1/tracks/006oGnrSZevq...,audio_features,spotify:track:006oGnrSZevqZTimsD2oh4,0.639
2,00Blm7zeNqgYLPtW6zg8cj,0.03610,https://api.spotify.com/v1/audio-analysis/00Bl...,Post Malone,0.687,193507.0,0.7810,00Blm7zeNqgYLPtW6zg8cj,0.000000,1.0,...,1.0,track/artist,0.0530,97.014,4.0,One Right Now (with The Weeknd),https://api.spotify.com/v1/tracks/00Blm7zeNqgY...,audio_features,spotify:track:00Blm7zeNqgYLPtW6zg8cj,0.688
3,00E0Z2jrF7reoHps4zcbWQ,0.34100,https://api.spotify.com/v1/audio-analysis/00E0...,Alok,0.719,147212.0,0.9590,00E0Z2jrF7reoHps4zcbWQ,0.000000,9.0,...,0.0,track/artist,0.1250,126.031,4.0,Car Keys (Ayla),https://api.spotify.com/v1/tracks/00E0Z2jrF7re...,audio_features,spotify:track:00E0Z2jrF7reoHps4zcbWQ,0.522
4,00KyYtT6NaXwbPecina5Pj,0.05280,https://api.spotify.com/v1/audio-analysis/00Ky...,Chris Young,0.535,166507.0,0.8590,00KyYtT6NaXwbPecina5Pj,0.000000,6.0,...,1.0,track/artist,0.0344,102.008,4.0,Famous Friends,https://api.spotify.com/v1/tracks/00KyYtT6NaXw...,audio_features,spotify:track:00KyYtT6NaXwbPecina5Pj,0.698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4104,7zFXmv6vqI4qOt4yGf3jYZ,0.42200,https://api.spotify.com/v1/audio-analysis/7zFX...,Daniel Caesar,0.658,278180.0,0.2940,7zFXmv6vqI4qOt4yGf3jYZ,0.000052,4.0,...,0.0,track/artist,0.0321,74.038,4.0,Get You (feat. Kali Uchis),https://api.spotify.com/v1/tracks/7zFXmv6vqI4q...,audio_features,spotify:track:7zFXmv6vqI4qOt4yGf3jYZ,0.358
4105,7zLYKWcXnYeHHWidalz7rj,0.46200,https://api.spotify.com/v1/audio-analysis/7zLY...,Lil Mosey,0.699,165000.0,0.5580,7zLYKWcXnYeHHWidalz7rj,0.000000,11.0,...,1.0,track/artist,0.0603,79.992,4.0,Noticed,https://api.spotify.com/v1/tracks/7zLYKWcXnYeH...,audio_features,spotify:track:7zLYKWcXnYeHHWidalz7rj,0.493
4106,7zVCrzzEJU7u24sbJPXA5W,0.00898,https://api.spotify.com/v1/audio-analysis/7zVC...,Kane Brown,0.612,188253.0,0.7990,7zVCrzzEJU7u24sbJPXA5W,0.000000,10.0,...,1.0,track/artist,0.0275,125.976,4.0,What Ifs (feat. Lauren Alaina),https://api.spotify.com/v1/tracks/7zVCrzzEJU7u...,audio_features,spotify:track:7zVCrzzEJU7u24sbJPXA5W,0.687
4107,7zkA4TSmYO1Bko2PnFn0YB,0.84100,https://api.spotify.com/v1/audio-analysis/7zkA...,L.Dre,0.676,134026.0,0.0674,7zkA4TSmYO1Bko2PnFn0YB,0.942000,7.0,...,1.0,track/artist,0.0703,77.037,4.0,Steven Universe,https://api.spotify.com/v1/tracks/7zkA4TSmYO1B...,audio_features,spotify:track:7zkA4TSmYO1Bko2PnFn0YB,0.214


In [ ]:
final_tracks_df.to_csv('/content/drive/MyDrive/2024_11_14_working_tracks_df.csv')

In [ ]:
tracks_df.to_csv('/content/drive/MyDrive/2024_11_14_tracks_df.csv')

NameError: name 'tracks_df' is not defined

In [ ]:
remaining_ids = tracks_df[tracks_df['danceability'].isnull()].index

In [ ]:
!touch /content/drive/MyDrive/rs_client_id.txt

In [ ]:
!touch /content/drive/MyDrive/rs_client_secret.txt

In [ ]:
# start from df
tracks_df = pd.read_csv('/content/drive/MyDrive/2024_11_14_tracks_df.csv')
remaining_ids = tracks_df[tracks_df['danceability'].isnull()]['Unnamed: 0']

In [ ]:
remaining_ids

,Unnamed: 0
88,6AQbmUe0Qwf5PZnt4HmTXv
149,3JpTsmtxg3lEHmQMkVYW2y
178,3F5CgOj3wFlRv51JsHbxhe
229,3h5TiWTqGxjSjFrbruPFH9
274,09SNuR471zu5nVvC4IVG8j
...,...
4104,240audWazVjwvwh7XwfSZE
4105,4ILc7H2NfGbzbrHBclNOtB
4106,01JMnRUs2YOK6DDpdQASGY
4107,0biCSADTAblvLTLtJz4pXO


In [ ]:
# @title Access Spotify API

# Read the client secret from a file
with open('/content/drive/MyDrive/rs_client_secret.txt', 'r') as file:
    client_secret = file.read().strip()  # Removes any extraneous whitespace or newlines

# Read the client id from a file
with open('/content/drive/MyDrive/rs_client_id.txt', 'r') as file:
    client_id = file.read().strip()  # Removes any extraneous whitespace or newlines

auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

headers = {
    "Authorization": f"Basic {b64_auth_str}"
}
data = {
    "grant_type": "client_credentials"
}


# Read the client secret from a file
with open('/content/drive/MyDrive/client_secret.txt', 'r') as file:
    client_secret = file.read().strip()  # Removes any extraneous whitespace or newlines

# Read the client id from a file
with open('/content/drive/MyDrive/client_id.txt', 'r') as file:
    client_id = file.read().strip()  # Removes any extraneous whitespace or newlines

auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()
response = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
access_token = response.json()['access_token']


In [ ]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

data = {
    "grant_type": "client_credentials"
}

audio_features_by_track={}
# Chunk the list of IDs into batches of 100
chunk_size = 100
remaining_ids_chunks = [remaining_ids[i:i + chunk_size] for i in range(0, len(remaining_ids), chunk_size)]

for chunk in remaining_ids_chunks:
    # Join the chunk of IDs into a comma-separated string
    ids = ','.join(chunk)

    # Make the request to the Audio Features endpoint with the batch of IDs
    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'
    response = requests.get(url, headers=headers)

    # Parse the JSON response
    if response.status_code == 200:
        audio_features_list = response.json().get('audio_features', [])
        for audio_features in audio_features_list:
            if audio_features:  # Some entries might be None if the ID is invalid
                track_id = audio_features['id']
                audio_features_by_track[track_id] = audio_features
                print(f"Retrieved audio features for Track ID {track_id}")
    else:
        print(f"Error: {response.status_code} - {response.json()}")

    # Sleep to avoid hitting rate limits
    time.sleep(1)  # Adjust as needed

Retrieved audio features for Track ID 6AQbmUe0Qwf5PZnt4HmTXv
Retrieved audio features for Track ID 3JpTsmtxg3lEHmQMkVYW2y
Retrieved audio features for Track ID 3F5CgOj3wFlRv51JsHbxhe
Retrieved audio features for Track ID 3h5TiWTqGxjSjFrbruPFH9
Retrieved audio features for Track ID 09SNuR471zu5nVvC4IVG8j
Retrieved audio features for Track ID 3CIyK1V4JEJkg02E4EJnDl
Retrieved audio features for Track ID 3lGBRoe3c6oV398Xwgf7c7
Retrieved audio features for Track ID 44uJJry70pJ0GZIzIGCbUn
Retrieved audio features for Track ID 2beS6dD5rovBxSA1zYoSHR
Retrieved audio features for Track ID 3e7sxremeOE3wTySiOhGiP
Retrieved audio features for Track ID 05bfbizlM5AX6Mf1RRyMho
Retrieved audio features for Track ID 39JofJHEtg8I4fSyo7Imft
Retrieved audio features for Track ID 1hjRhYpWyqDpPahmSlUTlc
Retrieved audio features for Track ID 6qNgUxNscSCV4Oqvo5Eo8T
Retrieved audio features for Track ID 0vbtURX4qv1l7besfwmnD8
Retrieved audio features for Track ID 7odvRjPaSvrPit3jmqaoHS
Retrieved audio features

In [ ]:
# headers = {
#     'Authorization': f'Bearer {access_token}'
# }

# data = {
#     "grant_type": "client_credentials"
# }

# audio_features_by_track={}
# num=0
# for id in remaining_ids:
#   num+=1
#   audio_features_by_track[id] = {}

#   # Make the request to the Audio Features endpoint
#   url = f'https://api.spotify.com/v1/audio-features/{id}'
#   response = requests.get(url, headers=headers)

#   # Parse the JSON response
#   if response.status_code == 200:
#       audio_features = response.json()
#       print("Audio Features for Track ID", id, num)
#       for feature, value in audio_features.items():
#           audio_features_by_track[id][feature] = value


#   else:
#       print(f"Error: {response.status_code} - {response.json()}")
#   time.sleep(1)  # Adjust as needed to stay within rate limits


Error: 429 - {'error': {'status': 429}}
Error: 429 - {'error': {'status': 429}}
Error: 429 - {'error': {'status': 429}}
Error: 429 - {'error': {'status': 429}}
Error: 429 - {'error': {'status': 429}}


In [ ]:
afdf=pd.DataFrame(audio_features_by_track).T

afdf.to_csv('/content/drive/MyDrive/audio_features_by_track.csv')

In [ ]:
response = requests.get('https://api.spotify.com/v1/audio-features/3r8RuvgbX9s7ammBn07D3W', headers=headers)

# Check rate limit headers
print(response.headers.get('x-ratelimit-remaining'))  # Remaining requests
print(response.headers.get('x-ratelimit-reset'))  # Reset time (Unix timestamp)

None
None


In [ ]:
response.json()

{'error': {'status': 429}}